# Model Training
- The notebook provides the code for training the scoring model in Experiment 1 and Experiment 2.

In [1]:
import torch
import torch.nn as nn
from utils.loader import Loader
from utils.model_arg import ModelArg
from utils.evaluate import correct, loss_func
from utils.plot_res import plot
from models.assess import Assess
from datetime import datetime
import pandas as pd
import pickle
import os
import random
import numpy as np

In [2]:
### Specify a worker ###
deviceId = 'cuda:0'
device = torch.device(deviceId)

In [3]:
def train(args, saveFlag=False, printFlag=True):
    """Training Block for the input data.
    :args: an instance of the ModelArgs class
    :saveFlag: whether to save best models
    :printFlag: whether to print out training logs
    """
    folder = datetime.now().strftime('%Y-%m-%d-%H%M%S')
    log_path = f"{args.model_save_to}{folder}"
    if saveFlag:
        os.mkdir(log_path)
    log_path += '/'
    torch.manual_seed(args.seed)
    train_loader, val_loader = Loader(args, device).load_dataset()
    torch.backends.cudnn.deterministic = True
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)
    model = Assess(device, param_dim=len(args.fields), linear_list=args.linear_list)
    model.to(device)
    parameter_sum = sum(p.numel() for p in model.parameters())
    if printFlag:
        print('Total Parameters: ', parameter_sum)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)     
    ### basic initialization
    train_len = len(train_loader)
    train_loss_log, train_acc_log, val_acc_log, score_diff = [], [], [], []
    max_acc, max_val_acc, max_delta = 0.0, 0.0, 0.0
    learning_rate = args.lr 
    optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
    ### training
    for epoch in range(args.epoch):
        total_loss = 0.0
        train_correct_sum, val_correct_sum = 0.0, 0.0
        train_current_len, val_current_len = 0.0, 0.0
        score_report = 0.0
        if epoch % 50 == 0:
            learning_rate /= 2
            optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)
        ### mini batch
        for i, (x, names, winningCount, lossingCount) in enumerate(train_loader):
            optimizer.zero_grad()
            model.train()
            score = model(x)
            score_report += torch.sum(torch.abs(score[0]-score[1]))
            loss = loss_func(score, (winningCount, lossingCount))
            loss.to(device)
            total_loss += loss
            train_correct_sum += correct(score)
            train_current_len += x[0].shape[0]
            loss.backward()
            optimizer.step()
        for i, (x, names, _, _) in enumerate(val_loader):
            model.eval()
            score = model(x)
            val_correct_sum += correct(score)
            val_current_len += x[0].shape[0]
        train_acc = train_correct_sum / train_current_len
        val_acc = val_correct_sum / val_current_len
        if printFlag:
            print(f'\rEpoch-{epoch}: Loss {total_loss :.4f}, Train Acc {train_acc:.3f}, Val Acc {val_acc:.3f}, δS {score_report:.2f}' + ' '*23)
        train_acc_log.append(train_acc.detach().cpu())
        train_loss_log.append(total_loss.detach().cpu())
        val_acc_log.append(val_acc.detach().cpu())
        score_diff.append(score_report.detach().cpu())
        #### save good models
        if max_acc <= train_acc:
            max_acc = train_acc
            if saveFlag:
                torch.save(model, f'{log_path}best_t.pt')
        if max_val_acc <= val_acc:
            max_val_acc = val_acc
            if saveFlag:
                torch.save(model, f'{log_path}best_v.pt')
        if max_delta <= score_report:
            max_delta = score_report
            if saveFlag:
                torch.save(model, f'{log_path}best_s.pt')
    logdata ={'train_loss': train_loss_log, 'train_acc': train_acc_log, 'val_acc': val_acc_log, 'score_diff': score_diff}
    ### save the training log and the hyper parameters
    if saveFlag:
        torch.save(model, f'{log_path}final.pt')
        with open(f'{log_path}loss_log_dict.pkl', 'wb') as f:
            pickle.dump(logdata, f)
        with open(f'{log_path}hyperparam.pkl', 'wb') as f:
            pickle.dump(args, f)
    return folder, logdata

## Exp 1

In [4]:
valAll = []
trainAll = []
n = 10
for monteCarloDVidx in range(n):   
    args = ModelArg(
        batch_size=512,
        lr=2,
        epoch=200,
        linear_list=[64, 32, 32, 16, 8, 4],
        val_split_ratio=0.2,
        label_path='../dataset/exp1/turk_results.csv',
        param_path='../dataset/exp1/parameters.csv',
        extreme_path='../dataset/exp1/extremes.csv',
        seed=monteCarloDVidx,
        fields=['width', 'nbar', 'bandwidth'],
        name=f'Experiment 1: {monteCarloDVidx}',
    )
    folder_name, log = train(args, saveFlag=False, printFlag=False)
    # plot(log, modelname=args.name, printFlag=True)
    trainAll.append(max(log['train_acc']))
    valAll.append(max(log['val_acc']))
print(f'Average accuracy after {n} rounds: {np.mean(trainAll):.4f} (Train) {np.mean(valAll):.4f} (Validation)')

Average accuracy after 10 rounds: 0.7593 (Train) 0.7560 (Validation)


## Exp2

In [5]:
valAll = []
trainAll = []
n = 1
for monteCarloDVidx in range(n):   
    args = ModelArg(
        batch_size=512,
        linear_list=[64, 256, 128, 64, 32, 16, 8, 4],
        val_split_ratio=0.2,
        label_path='../dataset/exp2/turk_results.csv',
        param_path='../dataset/exp2/parameters.csv',
        extreme_path='../dataset/exp2/extremes.csv',
        fields=['nbar', 'bandwidth', 'bandwidth', 'maxChar', 'rotation', 'orientation'],
        seed=monteCarloDVidx,
        epoch=200,
        lr=2,
        name=f'Experiment 2: {monteCarloDVidx}'
    )
    folder_name, log = train(args, saveFlag=False, printFlag=False)
    # plot(log, modelname=args.name, printFlag=True)
    trainAll.append(max(log['train_acc']))
    valAll.append(max(log['val_acc']))
print(f'Average accuracy after {n} rounds: {np.mean(trainAll):.4f} (Train) {np.mean(valAll):.4f} (Validation)')

Average accuracy after 1 rounds: 0.7762 (Train) 0.7743 (Validation)
